In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Categorical data

In [2]:
# The file has no headers naming the columns, so we pass header=None
# and provide the column names explicitly in "names"
data = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", na_values=[" ?"], 
    header=None, index_col=False,
    names=['age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'gender',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
    'income'])
# For illustration purposes, we only select some of the columns
data = data[['workclass', 'age', 'education', 'education-num', 'occupation', 'capital-gain','gender', 'hours-per-week',  'income']]
# IPython.display allows nice output formatting within the Jupyter notebook
# add some none
data['education-num'][0]=None
display(data.head())

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,workclass,age,education,education-num,occupation,capital-gain,gender,hours-per-week,income
0,State-gov,39,Bachelors,NaN,Adm-clerical,2174,Male,40,<=50K
1,Self-emp-not-inc,50,Bachelors,13.0,Exec-managerial,0,Male,13,<=50K
2,Private,38,HS-grad,9.0,Handlers-cleaners,0,Male,40,<=50K
3,Private,53,11th,7.0,Handlers-cleaners,0,Male,40,<=50K
4,Private,28,Bachelors,13.0,Prof-specialty,0,Female,40,<=50K


In [3]:
data = data[0:1000]

In [4]:
data.isnull().sum()

workclass         62
age                0
education          0
education-num      1
occupation        62
capital-gain       0
gender             0
hours-per-week     0
income             0
dtype: int64

Teraz rzućmy okiem na wszystkie atrybuty kategoryczne:

In [5]:
data["workclass"].value_counts()

 Private             698
 Self-emp-not-inc     81
 Local-gov            68
 State-gov            37
 Self-emp-inc         33
 Federal-gov          21
Name: workclass, dtype: int64

In [6]:
data["education"].value_counts()

 HS-grad         321
 Some-college    225
 Bachelors       166
 Masters          54
 Assoc-voc        48
 11th             46
 Assoc-acdm       35
 10th             21
 9th              16
 7th-8th          15
 Doctorate        14
 5th-6th          11
 Prof-school      10
 12th              9
 1st-4th           7
 Preschool         2
Name: education, dtype: int64

In [7]:
data["gender"].value_counts()

 Male      671
 Female    329
Name: gender, dtype: int64

In [8]:
data["occupation"].value_counts()

 Craft-repair         126
 Exec-managerial      124
 Prof-specialty       124
 Sales                112
 Other-service        107
 Adm-clerical          94
 Machine-op-inspct     61
 Transport-moving      52
 Tech-support          44
 Handlers-cleaners     43
 Farming-fishing       31
 Protective-serv       16
 Priv-house-serv        3
 Armed-Forces           1
Name: occupation, dtype: int64

Sprawdźmy, czy etykiety przyjmują wartości 0 lub 1.

Jak nie to musimy jes troszkę przerobić.

In [9]:
X = data.drop(['income'], axis=1)
y = data['income'].values
np.unique(y)
y[ y == ' <=50K'] = 0
y[ y == ' >50K'] = 1

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

print("X.shape: {} y.shape: {}".format(X.shape, y.shape))

X.shape: (1000, 8) y.shape: (1000,)


Podzielmy zbiór na train/test

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

Teraz zbudujmy nasze **pipeline** preprocessingu. 

Wykorzystamy DataframeSelector aby wybrać określone atrybuty z DataFrame:

In [36]:
from sklearn.base import BaseEstimator, TransformerMixin

# A class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

Zbudujmy **pipeline** dla atrybutów numerycznych:

In [76]:
# edu_dict = {
#     ' Preschool': 1.0, 
#     ' 1st-4th': 2.0, 
#     ' 5th-6th': 3.0,
#     ' 7th-8th': 4.0, 
#     ' 9th': 5.0, 
#     ' 10th': 6.0,
#     ' 11th': 7.0, 
#     ' 12th': 8.0, 
#     ' HS-grad': 9.0,
#     ' Some-college': 10.0, 
#     ' Assoc-voc': 11.0, 
#     ' Assoc-acdm': 12.0,
#     ' Bachelors': 13.0, 
#     ' Masters': 14.0
# }


# class EduEncoder(BaseEstimator, TransformerMixin):
#     def __init__(self, dictionary):
#         self.dictionary = dictionary
        
#     def fit(self, X, y=None):
#         return self
    
#     def transform(self, X, y=None):
#         a = X.values[:,1]
#         b = X.values[:,0]
#         ans = []
#         for i in np.arange( len(a)):
#             if np.isnan(a[i]):
#                 ans.append(edu_dict[b[i]])
#             else:    
#                 ans.append(a[i]) 
#         return ans

# edu_pipeline = Pipeline([
#         ("select_numeric", DataFrameSelector(["education","education-num"])),
#         ("dictionary_encoder", EduEncoder(edu_dict)),
#     ])

In [80]:
# class EduEncoder(BaseEstimator, TransformerMixin):
#     def __init__(self, dictionary):
#         self.dictionary = dictionary
        
#     def fit(self, X, y=None):
#         return self
    
#     def transform(self, X, y=None):
#         return X['education-num'].fillna(X['education'].apply(lambda x: edu_dict.get(x)))

# edu_pipeline = Pipeline([
#         ("select_numeric", DataFrameSelector(["education","education-num"])),
#         ("dictionary_encoder", EduEncoder(edu_dict)),
#     ])




In [81]:
edu_pipeline.fit_transform(X_train)

29      9.0
535     8.0
695    12.0
557     9.0
836     9.0
       ... 
106     6.0
270     9.0
860     7.0
435    14.0
102    14.0
Name: education-num, Length: 800, dtype: float64

In [64]:
# enumerate(edu_pipeline.fit_transform(X_train)["education-num"].values)

In [10]:
# a = edu_pipeline.fit_transform(X_train)["education-num"].values
# b = edu_pipeline.fit_transform(X_train)["education"].values
# ans = []
# for i in np.arange( len(a)):
#     if np.isnan(a[i]):
#         ans.append(edu_dict[b[i]])
#     else:    
#         ans.append(a[i])
# ans        

In [11]:
# a

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer



num_pipeline = Pipeline([
        ("select_numeric", DataFrameSelector(["education-num"])),
        ("imputer", SimpleImputer(strategy="median")),
    ])

In [13]:
num_pipeline.fit_transform(X_train)

array([[ 9.],
       [ 8.],
       [12.],
       [ 9.],
       [ 9.],
       [10.],
       [10.],
       [ 9.],
       [ 9.],
       [ 9.],
       [ 9.],
       [10.],
       [ 9.],
       [ 9.],
       [10.],
       [13.],
       [10.],
       [ 9.],
       [13.],
       [ 9.],
       [ 9.],
       [10.],
       [10.],
       [11.],
       [10.],
       [ 9.],
       [10.],
       [13.],
       [ 9.],
       [ 2.],
       [10.],
       [ 9.],
       [10.],
       [13.],
       [10.],
       [10.],
       [ 9.],
       [ 6.],
       [13.],
       [ 6.],
       [ 7.],
       [ 9.],
       [ 9.],
       [13.],
       [12.],
       [ 9.],
       [ 9.],
       [ 6.],
       [ 9.],
       [15.],
       [ 9.],
       [11.],
       [ 9.],
       [15.],
       [13.],
       [ 7.],
       [13.],
       [14.],
       [ 3.],
       [14.],
       [10.],
       [ 9.],
       [10.],
       [10.],
       [11.],
       [10.],
       [14.],
       [ 9.],
       [ 6.],
       [ 3.],
       [ 9.],
      

Będziemy także potrzebować imputera do kategorycznych kolumn napisowych (zwykły Imputer nie działa na tych kolumnach):

In [14]:
# Inspired from stackoverflow.com/questions/25239958
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

Teraz możemy zbudować pipeline dla atrybutów kategorycznych.

We can convert each categorical value to a one-hot vector using a OneHotEncoder. Right now this class can only handle integer categorical inputs, but in Scikit-Learn 0.20 it will also handle string categorical inputs (see PR https://github.com/scikit-learn/scikit-learn/issues/10521). So for now we import it from future_encoders.py

In [15]:
# from future_encoders import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

cat_pipeline = Pipeline([
        ("select_cat", DataFrameSelector(["workclass", "education", "occupation", "gender"])),
        ("imputer", MostFrequentImputer()),
        ("cat_encoder", OneHotEncoder(sparse=False, handle_unknown = 'ignore')),
    ])

In [16]:
cat_pipeline.fit_transform(X_train)

array([[0., 0., 1., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.]])

Na koniec połączmy powyższe podejścia:

In [17]:
from sklearn.pipeline import FeatureUnion
preprocess_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

In [18]:
preprocess_pipeline.fit_transform(X_train).shape

(800, 39)

In [19]:
y_train.shape

(800,)

# Zad

Robimy StratifiedKFold i znajdujemy optymalne parametry dla

* SVM z jądrem rbf
* SVM z jądrem poly
* SVM liniowego
* Regresji logistycznej

In [20]:
from sklearn.model_selection import StratifiedKFold

seed=123
kfold = StratifiedKFold(n_splits=5)

In [26]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
pipe = Pipeline([
    ('preprocessing', preprocess_pipeline), 
    ('scaler', StandardScaler()),
    ('classifier', SVC())])


param_grid = {
            'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
            'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100]    
}

grid_1 = GridSearchCV(pipe, param_grid, cv=kfold)

grid_1.fit(X_train, y_train)
grid_1.best_params_

{'classifier__C': 1, 'classifier__gamma': 1}

In [56]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

pipe = Pipeline([
    ('preprocessing', preprocess_pipeline), 
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(n_estimators=500, max_leaf_nodes=16))])


param_grid = {'classifier__n_estimators': [10, 50, 100],
              'classifier__max_leaf_nodes': [10, 20],
              'classifier__max_depth': [10, 20]
             }

grid_2 = GridSearchCV(pipe, param_grid, cv=kfold)

grid_2.fit(X_train, y_train)
grid_2.best_params_

{'classifier__max_depth': 10,
 'classifier__max_leaf_nodes': 10,
 'classifier__n_estimators': 50}

In [23]:
from sklearn import  metrics


models = []
models.append(('SVM ', grid_1.best_estimator_))



precision_score = []
recall_score = []
f1_score = []
accuracy_score = []
for name, model in models:
    print(name)
    print("precision_score: {}".format(metrics.precision_score(y_test, model.predict(X_test)) ))
    print("recall_score: {}".format( metrics.recall_score(y_test, model.predict(X_test)) ))
    print("f1_score: {}".format( metrics.f1_score(y_test, model.predict(X_test)) ))
    print("accuracy_score: {}".format( metrics.accuracy_score(y_test, model.predict(X_test)) ))
    precision_score.append(metrics.precision_score(y_test, model.predict(X_test)))
    recall_score.append(metrics.recall_score(y_test, model.predict(X_test)))
    f1_score.append( metrics.f1_score(y_test, model.predict(X_test)))
    accuracy_score.append(metrics.accuracy_score(y_test, model.predict(X_test)))

SVM linear
precision_score: 1.0
recall_score: 0.08333333333333333
f1_score: 0.15384615384615385
accuracy_score: 0.78


In [24]:
import pandas as pd
d = {'precision_score': precision_score, 
     'recall_score': recall_score, 
     'f1_score': f1_score,
     'accuracy_score' : accuracy_score
    }
df = pd.DataFrame(data=d)
df.insert(loc=0, column='Method', value=['SVM linear'])
df

,Method,precision_score,recall_score,f1_score,accuracy_score
0,SVM linear,1.0,0.083333,0.153846,0.78
